In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import seaborn as sns


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**I did EDA at other notebook:https://www.kaggle.com/imnaho/eda-tps-mar22**

**at this notebook I will create model and prediction for submit**

# read data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')
sample_submission=pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')
train.shape, test.shape,sample_submission.shape

# check data

In [ ]:
cols_with_missing = [col for col in train.columns if train[col].isnull().any()]
print('missing value in train data is:',cols_with_missing)
cols_with_missing = [col for col in test.columns if test[col].isnull().any()]
print('missing value in test data is:',cols_with_missing)

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
a=train.groupby(['x','y','direction']).mean()
a

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe(include='all')

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(train['congestion'])

In [ ]:
test.describe(include='all')

at EDA notebook I checked 
* train data and test data has no missing value
* train data has time 1991-04-01 00:00 ~ 1991-09-30 11:40  
* test data has time 1991-09-30 12:00 ~ 1991-09-30 23:40
* columns x,y and direction might be good feature
* week,hour also might be good feature from time column
* there are very little difference at each day

In [ ]:
for df in [train, test]:
    
    df['date'] = pd.to_datetime(df.time)
    le = LabelEncoder()
    encoded = le.fit_transform(df['direction'].values)
    df['direction']=encoded
    
    df['month']=df['date'].dt.month
    df['num_week']=df['date'].dt.week
    df['week']=df['date'].dt.weekday
    df['week_2']=0
    df.loc[df['week']==5,'week_2']=1
    df['day'] = df['date'].dt.day
    df["hour"] = df['date'].dt.strftime('%H').astype(int)
    df["minutes"] = df['date'].dt.strftime('%M').astype(int)
    df["num_of_time"] = df["hour"].astype(int)+df["minutes"]/60
    
    df["high_time_1"] = 0
    df.loc[df['hour']==8,"high_time_1"]=1
    df.loc[df['hour']==9,"high_time_1"]=1
    
    df["high_time_2"] = 0
    df.loc[df['hour']==16,"high_time_2"]=1
    df.loc[df['hour']==17,"high_time_2"]=1
    
    df["low_time"] = 0
    df.loc[df['hour']==4,"low_time"]=1
    df.loc[df['hour']==5,"low_time"]=1
    df.loc[df['hour']==6,"low_time"]=1
    
    df['month_end']=df['date'].dt.is_month_end
    df.loc[df['month_end']==True,'month_end']=1
    df.loc[df['month_end']==False,'month_end']=0
    
    df['quarter_start']=df['date'].dt.is_quarter_start
    df.loc[df['quarter_start']==True,'quarter_start']=1
    df.loc[df['quarter_start']==False,'quarter_start']=0
    
    df['quarter_end']=df['date'].dt.is_quarter_end
    df.loc[df['quarter_end']==True,'quarter_end']=1
    df.loc[df['quarter_end']==False,'quarter_end']=0
    
    df['month_end']=df['month_end'].astype(int)
    df['quarter_start']=df['quarter_start'].astype(int)
    df['quarter_end']=df['quarter_end'].astype(int)

train.head()

In [ ]:
train=train.drop(['row_id','time','date','hour','minutes'],axis=1)
test=test.drop(['row_id','time','date','hour','minutes'],axis=1)
print('train shape is:',train.shape)
print('test shape is:',test.shape)
train.dtypes

# create model

In [ ]:
#Separate data into training and validation sets
X=train.drop(['congestion'],axis = 1)
y = train.congestion

X_train_data, X_valid_data, y_train_data, y_valid_data = train_test_split(X, y, train_size=0.9, test_size=0.1,random_state=0)
my_model = XGBRegressor(n_estimators=2000, learning_rate=0.01,random_state=0)

In [ ]:
def fit_model(model,X_train_data_, y_train_data_,X_valid_data_, y_valid_data_):
    model.fit(X_train_data_, y_train_data_,early_stopping_rounds=5, 
              eval_set=[(X_valid_data_, y_valid_data_)],verbose=False,eval_metric='mae')
    prediction = model.predict(X_valid_data_)
    print('score is :',mean_absolute_error(y_valid_data_, prediction))

fit_model(my_model,X_train_data, y_train_data,X_valid_data, y_valid_data)

In [ ]:
plt.figure(figsize=(10,5))

importances = pd.Series(my_model.feature_importances_, index = X.columns)
importances = importances.sort_values()
importances.plot(kind = "barh")
plt.title("importance in the XGBRegressor Model")

plt.show()

# create prediction for submit

In [ ]:
df_prediction=my_model.predict(test)
sample_submission['congestion']=np.round(df_prediction)

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(sample_submission['congestion'])

In [ ]:
sample_submission.to_csv('submission.csv', index=False)